# Rasters and Tiles

Overview of basic types in GeoTrellis and their quirks. This workbook mostly tracks [Tiles](https://geotrellis.github.io/geotrellis-workshop/docs/tiles) from the workshop docs.



In [3]:
import $ivy.`org.slf4j:slf4j-simple:1.7.30`
import $ivy.`org.locationtech.geotrellis::geotrellis-raster:3.5.2`

import geotrellis.raster._

import $ivy.$                              

import $ivy.$                                                     


import geotrellis.raster._

- [`Tile`](https://github.com/locationtech/geotrellis/blob/master/raster/src/main/scala/geotrellis/raster/Tile.scala) is fundemental interface in GeoTrellis. 
- Most common type of `Tile` is [`ArrayTile`](https://github.com/locationtech/geotrellis/blob/master/raster/src/main/scala/geotrellis/raster/ArrayTile.scala) -- it is backed by Scala `Array` type.
- Any primitive type can back a `Tile` (`Int`, `Short`, `Double`, `Float`, `Byte`)
- While `Array`s are mutable, `Tile` interface does not allow mutation.
- Performance is primary concern behind the `Tile` interface design

In [4]:
val rawData = 1 to 16 toArray
val tile16 = ArrayTile(rawData, cols = 4, rows = 4)
Text(tile16.asciiDraw())

     1     2     3     4
     5     6     7     8
     9    10    11    12
    13    14    15    16



rawData: Array[Int] = Array(
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16
)
tile16: IntConstantNoDataArrayTile = IntConstantNoDataArrayTile(
  Array(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16),
  4,
  4
)

## CellType

Tile also has to have a [`CellType`](https://github.com/locationtech/geotrellis/blob/master/raster/src/main/scala/geotrellis/raster/CellType.scala) in order to carry the `NODATA` value and keep track of unsigned types. Its string representation encodes the nodata type.

- [NoDatas per data type](https://github.com/locationtech/geotrellis/blob/1a2ea84f7a15d790a13a75ede0fecee351ac4a7e/raster/src/main/scala/geotrellis/raster/package.scala#L104-L111)


In [5]:
tile16.cellType

res4: IntConstantNoDataCellType.type = int32

`NODATA` part of `CellType` effects what values we see:

In [6]:
tile16.get(0, 0)

res5: Int = 1

In [7]:
tile16.withNoData(Some(1)).get(0, 0)

res6: Int = -2147483648

Operations on `Tile` use `CellType` to determine "safe" result type.

In [10]:
IntCellType.union(FloatCellType)

res9: CellType = float32raw

## Tile Interface
Regardless of what type backs the Tile when we access the pixels values they're unified to either `Int` or `Double`. This is done for performance reasons and to facilitate working across cell types. Every method used to access pixel values has `_Double` version (ex: `get` and `getDouble`). The conversion happens on the fly, without affecting the underlying data.

In [12]:
tile16.get(0,0)
tile16.getDouble(0,0)

res11_0: Int = 1
res11_1: Double = 1.0

In [18]:
val floats = (1 to 16 toArray).map(x => x.toFloat / 10)
val tileFloat = ArrayTile(floats, cols = 4, rows = 4)
Text(tileFloat.asciiDraw)

     0     0     0     0
     0     0     0     0
     0     1     1     1
     1     1     1     1



floats: Array[Float] = Array(
  0.1F,
  0.2F,
  0.3F,
  0.4F,
  0.5F,
  0.6F,
  0.7F,
  0.8F,
  0.9F,
  1.0F,
  1.1F,
  1.2F,
  1.3F,
  1.4F,
  1.5F,
  1.6F
)
tileFloat: FloatConstantNoDataArrayTile = FloatConstantNoDataArrayTile(
  Array(
    0.1F,
    0.2F,
    0.3F,
    0.4F,
    0.5F,
    0.6F,
    0.7F,
    0.8F,
    0.9F,
    1.0F,
    1.1F,
    1.2F,
    1.3F,
    1.4F,
    1.5F,
    1.6F
  ),
  4,
  4
)

In [19]:
tileFloat.get(0,0)
tileFloat.getDouble(0,0)

res18_0: Int = 0
res18_1: Double = 0.10000000149011612

### Tile Interface and NODATA

When using `Tile` methods the `NODATA` cells are converted to either `Int.MinValue` or `Double.NaN` 

**regardless of underlying cell value**

In [31]:
val tile5ND = tile16.withNoData(Some(5))

tile5ND: Tile = IntUserDefinedNoDataArrayTile(
  Array(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16),
  4,
  4,
  IntUserDefinedNoDataCellType(5)
)

In [21]:
Text(tile5ND.asciiDraw())

     1     2     3     4
    ND     6     7     8
     9    10    11    12
    13    14    15    16



In [30]:
tile5ND.get(0, 1)
tile5ND.getDouble(0, 1)

res29_0: Int = -2147483648
res29_1: Double = NaN

Use `isData` and `isNoData` macros to test cell values from `Tile`. 

They're macros that are valid for both `Int` and `Double` values.

In [29]:
var intCount = 0

tile5ND.foreach { v => if (isData(v)) intCount += 1 }

intCount: Int = 15

In [24]:
val tileFND = tileFloat.withNoData(Some(0.5F))

tileFND: FloatArrayTile = FloatUserDefinedNoDataArrayTile(
  Array(
    0.1F,
    0.2F,
    0.3F,
    0.4F,
    0.5F,
    0.6F,
    0.7F,
    0.8F,
    0.9F,
    1.0F,
    1.1F,
    1.2F,
    1.3F,
    1.4F,
    1.5F,
    1.6F
  ),
  4,
  4,
  FloatUserDefinedNoDataCellType(0.5F)
)

In [27]:
Text(tileFND.asciiDrawDouble(1))

   0.1   0.2   0.3   0.4
    ND   0.6   0.7   0.8
   0.9     1     1     1
     1     1     2     2



In [32]:
var floatCount = 0

tileFND.foreach { v => if (isData(v)) floatCount += 1 }

floatCount: Int = 15

### Actual way to use Tile

In practice you almost never want to use `Tile.get`, there are better ways to do work

In [34]:
// immutability!
tile5ND.map { v => v + 1 }
// Hey, value at index 5 changed ...

res33_0: Tile = IntUserDefinedNoDataArrayTile(
  Array(2, 3, 4, 5, -2147483647, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17),
  4,
  4,
  IntUserDefinedNoDataCellType(5)
)
res33_1: Tile = IntUserDefinedNoDataArrayTile(
  Array(2, 3, 4, 5, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17),
  4,
  4,
  IntUserDefinedNoDataCellType(5)
)

In [35]:
tile16.combine(tileFloat) { (a, b) => a + b }

res34: ArrayTile = FloatConstantNoDataArrayTile(
  Array(
    1.0F,
    2.0F,
    3.0F,
    4.0F,
    5.0F,
    6.0F,
    7.0F,
    8.0F,
    9.0F,
    11.0F,
    12.0F,
    13.0F,
    14.0F,
    15.0F,
    16.0F,
    17.0F
  ),
  4,
  4
)

In [36]:
tile16.combineDouble(tileFloat) { (a, b) => a + b }

res35: ArrayTile = FloatConstantNoDataArrayTile(
  Array(
    1.1F,
    2.2F,
    3.3F,
    4.4F,
    5.5F,
    6.6F,
    7.7F,
    8.8F,
    9.9F,
    11.0F,
    12.1F,
    13.2F,
    14.3F,
    15.4F,
    16.5F,
    17.6F
  ),
  4,
  4
)

In [88]:
tile16 + tileFloat // hey, that's safe!

res87: Tile = FloatConstantNoDataArrayTile(
  Array(
    667.1F,
    2.2F,
    3.3F,
    4.4F,
    5.5F,
    6.6F,
    7.7F,
    8.8F,
    9.9F,
    11.0F,
    12.1F,
    13.2F,
    14.3F,
    15.4F,
    16.5F,
    17.6F
  ),
  4,
  4
)

### Mutable Tiles

In [90]:
val mut = tile16.mutable
mut.set(0, 0, 667)

mut: MutableArrayTile = IntConstantNoDataArrayTile(
  Array(667, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16),
  4,
  4
)

## Rasters

[`Raster`](https://github.com/locationtech/geotrellis/blob/2f8348ac299d889282b7e6d379eed4696ece1dd7/raster/src/main/scala/geotrellis/raster/Raster.scala#L63) is a composition of `Tile` and `Extent`, placing the tile pixels on a map.

It's up to you to know what projection that raster is in, it does not track `CRS`.

In [37]:
import geotrellis.vector._

val raster16 = Raster(tile16, Extent(0, 0, 2, 2))

import geotrellis.vector._


raster16: Raster[IntConstantNoDataArrayTile] = Raster(
  IntConstantNoDataArrayTile(
    Array(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16),
    4,
    4
  ),
  Extent(0.0, 0.0, 2.0, 2.0)
)

In [ ]:
raster16.cellSize

### RasterExtent

`Raster.rasterExtent` helps you translate between pixel space and map space

In [38]:
val re = raster16.rasterExtent

re: RasterExtent = RasterExtent(Extent(0.0, 0.0, 2.0, 2.0), 0.5, 0.5, 4, 4)

In [ ]:
re.gridToMap(0, 0)
re.gridToMap(0, 3)

In [ ]:
re.mapToGrid(0.3, 0.3) // closest cell

You can even rasterizer a geometry:
- [`RasterExtentRasterizeMethods`](https://github.com/locationtech/geotrellis/blob/master/raster/src/main/scala/geotrellis/raster/rasterize/RasterExtentRasterizeMethods.scala)
- available via [implicit method extension](https://github.com/locationtech/geotrellis/blob/1a2ea84f7a15d790a13a75ede0fecee351ac4a7e/raster/src/main/scala/geotrellis/raster/rasterize/Implicits.scala#L26)

In [39]:
val line = LineString(List(Point(0,0), Point(2, 2)))

re.foreach(line){ (col, row) => println(s"$col, $row") }

1, 3
2, 2
3, 1


line: LineString = LINESTRING (0 0, 2 2)